In [23]:
# import packages
import numpy as np
import pandas as pd
from gsheets import Sheets

In [29]:
# import sheets
sheets = Sheets.from_files('client_secret.json', '~/storage.json')
url = "https://docs.google.com/spreadsheets/d/1HyxulsjlOkI5Ike5KlcrEgik5Y6_XY_wVcKP6Y-eMpM/edit?usp=sharing"
s = sheets.get(url)

In [3]:
# make dataframe, change column types
s.sheets[0].to_csv(make_filename="sheet1.csv")
df = pd.read_csv('sheet1.csv')
df.female = df.female.astype("float32")
df.male = df.male.astype("float32")

In [4]:
# check typos in Family
df.Druzina.drop_duplicates().sort_values().tolist()

['Agelenidae',
 'Araneidae',
 'Atypidae',
 'Cheirachantiidae',
 'Clubionidae',
 'Dictynidae',
 'Gnaphosidae',
 'Linyphiidae',
 'Lycosidae',
 'Mimetidae',
 'Miturgidae',
 'Oxyopidae',
 'Philodromidae',
 'Pholcidae',
 'Phrurolithidae',
 'Pisauridae',
 'Salticidae',
 'Segestriidae',
 'Tetragnathidae',
 'Theridiidae',
 'Thomisidae',
 'Uloboridae',
 'Zodariidae']

In [5]:
# check typos in Genus
df.Rod.drop_duplicates().sort_values().tolist()

['Agelena',
 'Agyneta',
 'Allagelena',
 'Araneus',
 'Arctosa',
 'Argiope',
 'Attulus',
 'Atypus',
 'Ballus',
 'Bathyphantes',
 'Brigittea',
 'Cheiracanthium',
 'Clubiona',
 'Diplostyla',
 'Dolomedes',
 'Enoplognatha',
 'Erigone',
 'Ero',
 'Euryopis',
 'Evarcha',
 'Floronia',
 'Frontinellina',
 'Gibbaranea',
 'Heliophanus',
 'Hogna',
 'Holocnemus',
 'Hylyphantes',
 'Hypsosinga',
 'Hyptiotes',
 'Larinioides',
 'Leviellus',
 'Linyphia',
 'Mangora',
 'Marpissa',
 'Mendoza',
 'Mermesus',
 'Meta',
 'Metellina',
 'Microneta',
 'Misumena',
 'Neon',
 'Neriene',
 'Nuctenea',
 'Oxyopes',
 'Ozyptila',
 'Palliduphantes',
 'Panamomops',
 'Parasteatoda',
 'Pardosa',
 'Philodromus',
 'Pholcus',
 'Phrurolithus',
 'Phylloneta',
 'Pirata',
 'Piratula',
 'Pisaura',
 'Porrhomma',
 'Scotophaeus',
 'Segestria',
 'Steatoda',
 'Synema',
 'Tegenaria',
 'Tenuiphantes',
 'Tetragnatha',
 'Theridion',
 'Tibellus',
 'Tmarus',
 'Trochosa',
 'Walckenaria',
 'Xerolycosa',
 'Xysticus',
 'Zelotes',
 'Zodarion',
 'Zora',


In [ ]:
# check typos in Species
df.Vrsta.drop_duplicates().sort_values().tolist()

In [6]:
# define function for cleaning dataframes
def clean_wrt_species(df):
    df["Vrsta"] = df["Vrsta"].fillna("sp.")
    species_filter = "not Vrsta.str.startswith('sp.') and not Vrsta.str.startswith('cf.')"
    df.query(species_filter, inplace=True)
    return df

In [7]:
# define function for counting no. of unique family, genus and species
def spider_stats(df):
    family_no = df.Druzina.nunique() 
    genus_no = df.Rod.nunique()    
    species_no = df[["Rod", "Vrsta"]].drop_duplicates().shape[0]
    print(f'{family_no} družin, {genus_no} rodov, {species_no} vrst')

In [8]:
# summarise the stats
clean_df = clean_wrt_species(df)
spider_stats(clean_df)

17 družin, 57 rodov, 80 vrst


In [9]:
# define function for making etiquette for final reports
def make_etiquette(df):
    
    results = []
    for _, row in df.iterrows():    
        gender_elements = []
        if not np.isnan(row.female):
            gender_elements.append(f"{int(row.female)}F")

        if not np.isnan(row.male):
            gender_elements.append(f"{int(row.male)}M")

        results.append(f'{row.etiketa} ({", ".join(gender_elements)})')
    return ", ".join(results)


ser = clean_df.groupby(["Druzina", "Rod", "Vrsta"]).apply(make_etiquette)
ser.name = 'lokalitete'
ndf = ser.reset_index()

ndf

,Druzina,Rod,Vrsta,lokalitete
0,Agelenidae,Agelena,labyrinthica,"RTSB22-02 (2F), RTSB22-12 (1F), RTSB22-13 (1F)..."
1,Agelenidae,Allagelena,gracilens,"RTSB22-02 (2M), RTSB22-16 (1F), RTSB22-21 (1M)"
2,Araneidae,Araneus,angulatus,RTSB22-21 (1F)
3,Araneidae,Araneus,diadematus,RTSB22-28 (1M)
4,Araneidae,Araneus,marmoreus,"RTSB22-02 (1F), RTSB22-15 (1F)"
...,...,...,...,...
75,Theridiidae,Theridion,pinastri,RTSB22-01 (1F)
76,Thomisidae,Misumena,vatia,"RTSB22-02 (2M), RTSB22-07 (4M), RTSB22-10 (1M)..."
77,Thomisidae,Ozyptila,praticola,RTSB22-18 (1F)
78,Thomisidae,Synema,globosum,RTSB22-07 (1F)


In [10]:
# check for no M,F content
mask = ["()" in lok for lok in ndf.lokalitete]
ndf.loc[mask]

,Druzina,Rod,Vrsta,lokalitete


In [11]:
# setup for error checkup
rv_vals = [" ".join(x) for x in clean_df[["Rod", "Vrsta"]].drop_duplicates().values]
drv_vals = [" ".join(x[1:]) for x in clean_df[["Druzina", "Rod", "Vrsta"]].drop_duplicates().values]
v1, c1 = np.unique(rv_vals, return_counts=True)
v2, c2 = np.unique(drv_vals, return_counts=True)

In [13]:
# compare the two, to check for human error (wrong family)
mask = (clean_df.Rod + " " + clean_df.Vrsta).isin(v1[np.where(c1 != c2)[0]])
clean_df.loc[mask][["Druzina", "Rod", "Vrsta"]].value_counts()

Series([], dtype: int64)

In [28]:
# load csv, import in GSheets file
ndf.to_csv("test.csv", index=False)